In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/question-translation/questions-translation.csv
/kaggle/input/translated-questions/translated_questions.csv
/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/train_spider.json
/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/train_others.json
/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/dev_gold.sql
/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/dev.json
/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/README.txt
/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/train_gold.sql
/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/tables.json
/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/database/solvency_ii/solvency_ii.sqlite
/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/database/solvency_ii/schema.sql
/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/database/voter_1/voter_1.sqlite
/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/database/concert_si

In [2]:
pip install transformers sentencepiece sacremoses


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 14.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install torch transformers bitsandbytes accelerate sqlparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 21.7 MB/s eta 0:00:00:00:0100:01


In [4]:
!pip install torch transformers bitsandbytes accelerate sqlparse 

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re

model_name = "Rahmaa33/MotherDuckTEXT2SQLLArabicc"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    use_cache=True,
)

tokenizer_config.json:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

In [5]:
import pandas as pd
translation_path = "/kaggle/input/question-translation/questions-translation.csv"
df = pd.read_csv(translation_path)
df.head()

,question,translation
0,How many heads of the departments are older th...,كم رئيسًا للأقسام هم أكبر من 56 سنة؟
1,"List the name, born state and age of the heads...",قائمة بأسماء رؤساء الأقسام، مكان ميلادهم، وأعم...
2,"List the creation year, name and budget of eac...",قائمة بسنوات الإنشاء، وأسماء وميزانيات كل قسم.
3,What are the maximum and minimum budget of the...,ما هي أقصى وأدنى ميزانية للأقسام؟
4,What is the average number of employees of the...,ما هو المتوسط ​​لعدد الموظفين في الأقسام الذين...


In [6]:
import re
def process(record):
    if not isinstance(record['translation'], str):
        return record
    
    record['translation'] = re.sub(re.compile(r'\[Translation]\s*'), '', record['translation'])
    record['translation'] = re.sub(re.compile(r'\[Question]\s*'), '', record['translation'])
    return record
df_processed = df.apply(process, axis=1)
df_processed.head()

,question,translation
0,How many heads of the departments are older th...,كم رئيسًا للأقسام هم أكبر من 56 سنة؟
1,"List the name, born state and age of the heads...",قائمة بأسماء رؤساء الأقسام، مكان ميلادهم، وأعم...
2,"List the creation year, name and budget of eac...",قائمة بسنوات الإنشاء، وأسماء وميزانيات كل قسم.
3,What are the maximum and minimum budget of the...,ما هي أقصى وأدنى ميزانية للأقسام؟
4,What is the average number of employees of the...,ما هو المتوسط ​​لعدد الموظفين في الأقسام الذين...


In [7]:
import json
spider_db_path = "/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/database"
spider_train_path = "/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/train_spider.json"

with open(spider_train_path, "r") as f:
    data = json.load(f)
    
data[0]

{'db_id': 'department_management',
 'query': 'SELECT count(*) FROM head WHERE age  >  56',
 'query_toks': ['SELECT',
  'count',
  '(',
  '*',
  ')',
  'FROM',
  'head',
  'WHERE',
  'age',
  '>',
  '56'],
 'query_toks_no_value': ['select',
  'count',
  '(',
  '*',
  ')',
  'from',
  'head',
  'where',
  'age',
  '>',
  'value'],
 'question': 'How many heads of the departments are older than 56 ?',
 'question_toks': ['How',
  'many',
  'heads',
  'of',
  'the',
  'departments',
  'are',
  'older',
  'than',
  '56',
  '?'],
 'sql': {'except': None,
  'from': {'conds': [], 'table_units': [['table_unit', 1]]},
  'groupBy': [],
  'having': [],
  'intersect': None,
  'limit': None,
  'orderBy': [],
  'select': [False, [[3, [0, [0, 0, False], None]]]],
  'union': None,
  'where': [[False, 3, [0, [0, 10, False], None], 56.0, None]]}}

In [9]:
import os
import sqlite3
import torch
import re

# You should define the path to your Spider databases here.
# For example: spider_db_path = "/path/to/spider/databases"

def get_conn(db_id: str):
    """
    Establish a connection to the SQLite database specified by db_id.
    """
    db_file = f"{db_id}.sqlite"
    db_path = os.path.join(spider_db_path, db_id, db_file)
    conn = sqlite3.connect(db_path)
    return conn

def get_db_schema(conn) -> str:
    """
    Retrieve and concatenate the SQL creation statements for all tables
    in the connected SQLite database, effectively yielding its schema.
    """
    res = conn.execute("SELECT * FROM sqlite_master").fetchall()
    schema = ""
    for d in res:
        if d[-1] is None:
            continue
        schema += f"{d[-1]}\n\n"
    return schema

def inference(question: str, schema: str) -> str:
    """
    Generate an SQL query that answers the given question, based on a specified SQLite schema.

    This function uses a pre-trained text generation model and tokenizer (assumed to be
    defined globally or in the current scope as `model` and `tokenizer`) to produce an SQL
    query. The resulting query is then adapted from potential PostgreSQL-specific syntax
    to SQLite-compatible syntax.

    Args:
        question (str): The natural language question to convert to an SQL query.
        schema (str): The SQLite database schema, provided as a string.

    Returns:
        str: A single SQL query (as a string) that can be run against the specified SQLite database.
    """
    # Construct the prompt used by the model.
    prompt = (
        "### Examples\n"
        "Some example questions and corresponding SQL queries are provided based on similar problems:\n"
        "Answer the following: ما هو عدد المستخدمين المسجلين في التطبيق؟\n"
        "SELECT COUNT(*) FROM users;\n\n"
        "### Instructions:\n"
        "Your task is convert a question into a SQL query, given a sqlite database schema.\n"
        "Adhere to these rules:\n"
        "- **Deliberately go through the question and database schema word by word** to appropriately answer the question\n"
        "- **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.\n"
        "- When creating a ratio, always cast the numerator as float\n"
        "- **Use only one column per sql query**\n\n"
        "### Input:\n"
        f"Generate a SQL query that answers the question `{question}`.\n"
        "This query will run on a database whose schema is represented in this string:\n"
        f"{schema}\n"
        "### Response:\n"
        f"Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:\n"
        "```sql\n"
    )

    # These variables (model and tokenizer) need to be defined or imported from your environment.
    eos_token_id = tokenizer.eos_token_id
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate the SQL query from the model.
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=400,
        do_sample=False,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    # Clear CUDA cache and synchronize (optional but sometimes helpful).
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

    def convert_postgres_to_sqlite(query_text: str) -> str:
        """
        Convert any PostgreSQL-specific syntax in the query to be SQLite-compliant.

        Args:
            query_text (str): The SQL query string to be modified.

        Returns:
            str: A version of the query compatible with SQLite.
        """
        substitutions = [
            (r'ilike', 'LIKE'),
            (r'serial\s*$', 'INTEGER PRIMARY KEY AUTOINCREMENT'),
            (r'start\s+with\s+(\d+)', 'CHECK (id >= \\1)'),
        ]

        for pattern, replacement in substitutions:
            query_text = re.sub(pattern, replacement, query_text, flags=re.IGNORECASE)
        return query_text

    # Extract the query content from the generated output.
    # The generated text may look like: "...```sql\nSELECT ...\n```".
    postgres_query = outputs[0].split("```sql")[-1].rstrip("```")

    # Convert potential PostgreSQL-specific syntax to SQLite.
    query = convert_postgres_to_sqlite(postgres_query)

    return query


In [10]:
def compare_sql(gold: str, gen: str, conn, is_ordered=False):
    try:
        gold_res = pd.read_sql(gold, conn)
    except Exception as _:
        print("[Ground Fail]", gold)
        return 1

    try:
        gen_res = pd.read_sql(gen, conn)
    except Exception as _:
        print("[Gen Fail]", gen)
        return 0

    accuracy = 0
    if (len(gold_res)) == 0:
        return 1 if len(gen_res) == 0 else 0
    
    gold_len = len(gold_res)
    gen_len = len(gen_res)
    for i in range(min(gold_len, gen_len)):
        gold_record = gold_res.values[i]
        
        if not is_ordered:
            try:
                is_match = gold_record in gen_res.values
            except:
                is_match = False
        else:
            is_match = gold_record == gen_res.values[i]
            if (type(is_match) != bool):
                is_match = is_match.all()
                
        if is_match:
            accuracy += 1

    return accuracy / len(gold_res)

In [11]:
import sqlparse
from transformers import MarianMTModel, MarianTokenizer

# Load Marian NMT model for English-Arabic translation
def translate(text: str, src_lang="ar", tgt_lang="en") -> str:
    """Translate Arabic text to English using Marian NMT (Open Source)."""
    model_name = f"Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

In [12]:
import sqlparse
import nltk
from nltk.translate.bleu_score import sentence_bleu

# Initialize BLEU score
nltk.download('punkt')

# Evaluation parameters
NUM_EVAL = 100
total_accuracy = 0
total_execution_accuracy = 0
total_bleu_score = 0

for i, h in enumerate(data):
    if i >= NUM_EVAL:
        break
        
    db_id = h['db_id']
    conn = get_conn(db_id)  # Fetch database connection
    schema = get_db_schema(conn)  # Get database schema

    # Use Marian NMT for Arabic-English translation
    arabic_question = df.iloc[i]['translation']
    english_translation = translate(arabic_question)

    # Generate SQL query
    gen = inference(english_translation, schema)
    gold = h['query']



    # Execute queries and compare results (Execution Accuracy)
    execution_accuracy = compare_sql(gold, gen, conn)
    total_execution_accuracy += execution_accuracy

    # Compute BLEU score
    # Tokenize the SQL queries into words for BLEU evaluation
    gold_tokens = nltk.word_tokenize(gold)
    gen_tokens = nltk.word_tokenize(gen)
    bleu_score = sentence_bleu([gold_tokens], gen_tokens)
    total_bleu_score += bleu_score

    # Print evaluation results
    format_gold = sqlparse.format(gold, reindent=True)
    format_gen = sqlparse.format(gen, reindent=True)

    print(f"[{i}] Accuracy: {execution_accuracy * 100}%\n> Question: {english_translation}\n> Ground:\n{format_gold}\n\n> Gen:\n{format_gen}\n")
    print(f"BLEU Score: {bleu_score}\n")

# Compute final accuracy
final_accuracy = total_accuracy / NUM_EVAL
final_execution_accuracy = total_execution_accuracy / NUM_EVAL
final_bleu_score = total_bleu_score / NUM_EVAL

print(f"Final Execution Accuracy: {final_execution_accuracy * 100}%")
print(f"Final BLEU Score: {final_bleu_score}")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

...

[91] Accuracy: 100.0%
> Question: Looking for different cities for people's addresses?
> Ground:
SELECT DISTINCT T1.city
FROM addresses AS T1
JOIN people_addresses AS T2 ON T1.address_id = T2.address_id

> Gen:

SELECT DISTINCT T1.city
FROM Addresses AS T1
JOIN People_Addresses AS T2 ON T1.address_id = T2.address_id;

BLEU Score: 0.537284965911771

[92] Accuracy: 100.0%
> Question: What different cities do people live in?
> Ground:
SELECT DISTINCT T1.city
FROM addresses AS T1
JOIN people_addresses AS T2 ON T1.address_id = T2.address_id

> Gen:

SELECT DISTINCT T1.city
FROM Addresses AS T1
JOIN People_Addresses AS T2 ON T1.address_id = T2.address_id;

BLEU Score: 0.537284965911771

[93] Accuracy: 0.0%
> Question: Looking for different cities for student addresses?
> Ground:
SELECT DISTINCT T1.city
FROM addresses AS T1
JOIN people_addresses AS T2 ON T1.address_id = T2.address_id
JOIN students AS T3 ON T2.person_id = T3.student_id

> Gen:

SELECT DISTINCT T1.city
FROM Addresses AS T1
JOIN

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


[95] Accuracy: 100.0%
> Question: Show a list of course names in alphabetical order?
> Ground:
SELECT course_name
FROM courses
ORDER BY course_name

> Gen:

SELECT course_name
FROM Courses
ORDER BY course_name;

BLEU Score: 0.6147881529512643

[96] Accuracy: 100.0%
> Question: What are the names of the courses in alphabetical order?
> Ground:
SELECT course_name
FROM courses
ORDER BY course_name

> Gen:

SELECT course_name
FROM Courses
ORDER BY course_name;

BLEU Score: 0.6147881529512643

[97] Accuracy: 100.0%
> Question: A list of first names in alphabetical order?
> Ground:
SELECT first_name
FROM people
ORDER BY first_name

> Gen:

SELECT first_name
FROM People
ORDER BY first_name;

BLEU Score: 0.6147881529512643



Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


[98] Accuracy: 100.0%
> Question: What are the names of the first people in alphabetical order?
> Ground:
SELECT first_name
FROM people
ORDER BY first_name

> Gen:

SELECT first_name,
       last_name
FROM People
ORDER BY first_name,
         last_name;

BLEU Score: 0.34172334076593075

[99] Accuracy: 100.0%
> Question: What are the identification numbers of students who have recorded or attended courses?
> Ground:
SELECT student_id
FROM student_course_registrations
UNION
SELECT student_id
FROM student_course_attendance

> Gen:

SELECT student_id
FROM Student_Course_Registrations
UNION
SELECT student_id
FROM Student_Course_Attendance;

BLEU Score: 0.37991784282579627

Final Execution Accuracy: 69.65%
Final BLEU Score: 0.4698313105174533
